# Adiabatic SSM model

## 2. Scattered interpolation

This notebook takes multiple local SSM models which have been identified at pre-tensioned equilibria and interpolates them using scattered interpolation to obtain one adiabatic SSM model. The model performance is evaluated by its ability to predict open-loop trajectories.

In [1]:
from os import listdir
from os.path import join, isdir
import pickle
import yaml
import numpy as np
np.set_printoptions(linewidth=100)

In [2]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt

In [12]:
# SETTINGS
ROMOrder = 1
N_samples = 100
DT = 0.01
INTERPOLATION_METHOD = "linear" #  "idw" # "nn" # "origin_only", "linear", "ct", "nn"
ORIGIN_ONLY = (INTERPOLATION_METHOD == "origin_only")

In [13]:
observables = "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = join(robot_dir, 'rest_qv.pkl')

In [14]:
# directory containing the SSM models
data_dir = f"/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic_N={N_samples}" # {int(DT*1000)}ms_
model_names = [name for name in sorted(listdir(data_dir)) if isdir(join(data_dir, name))]
print(model_names)
N_models = len(model_names)

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017']


Load the pre-tensioned equilibrium of each model and plot adiabatic SSM map

In [88]:
models = []
test_results = []
observables = "delay-embedding"
for model_name in model_names:
    model_dir = join(data_dir, model_name, f"SSMmodel_{observables}") # _ROMOrder={ROMOrder}")
    with open(join(model_dir, "SSM_model.pkl"), "rb") as f:
        model = pickle.load(f)
        models.append(model)
    with open(join(model_dir, "test_results.yaml"), "rb") as f:
        test_results_dict = yaml.safe_load(f)
        test_results.append([test_results_dict['like training data']['RMSE'], test_results_dict['open-loop_circle']['RMSE']])
V = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
B_r = [model['model']['B'] for model in models]
q_bar = [model['model']['q_eq'] for model in models]
u_bar = [model['model']['u_eq'] for model in models]
ROMOrder = models[0]['params']['ROM_order']
SSMOrder = models[0]['params']['SSM_order']
for model in models:
    assert model['params']['ROM_order'] == ROMOrder
    assert model['params']['SSM_order'] == SSMOrder
test_results = np.array(test_results)

In [90]:
# for model in models:
#     print(model['model']['V'][2, 5])

In [21]:
# xyz_rest = np.array([q_bar[i][:2] for i in range(N_models)])
with open(join(data_dir, "rest_qs.pkl"), "rb") as f:
    xyz_rest = pickle.load(f)
xyz_rest = np.array(xyz_rest)[:, 3*TIP_NODE:3*TIP_NODE+3]
xyz_rest

array([[  3.58879936,   0.29345365, 195.6144321 ],
       [-27.40050589, -36.68798979, 180.09842115],
       [-14.60346578,   7.4295816 , 183.85632844],
       [ -6.11980062,   7.53423251, 187.14710148],
       [ 10.64656293, -44.13471862, 177.24710812],
       [ 61.75544639, -59.81437188, 157.74698746],
       [-14.46030436,  16.95554575, 182.76490951],
       [-41.06139362,  57.11783935, 173.95550417],
       [-28.74410817,  56.64576793, 170.93730663],
       [ 29.39531707, -60.74868375, 173.81116797],
       [ 23.79314212,  59.47637974, 150.17543467],
       [  6.36561914,  86.79224203, 150.04646726],
       [-56.14350794, -58.57027772, 134.67263502],
       [-75.2799343 ,  -7.4447859 , 159.33011031],
       [ 30.93280333, -13.10131213, 188.959437  ],
       [ 42.31195702,  34.3957865 , 145.08682448],
       [-26.64640245, -24.82785272, 181.4297449 ],
       [-13.80538936, -25.04052192, 181.5684651 ],
       [ 30.98541103,  21.00993603, 177.33147113],
       [ 22.10301809, -20.18737

Compute interpolations

In [92]:
from interpolators import InterpolatorFactory
coeff_dict = {
            'w_coeff': w_coeff,
            'V': V,
            'r_coeff': r_coeff,
            'B_r': B_r,
            'u_bar': u_bar,
            'q_bar': q_bar
        }
interpolator = InterpolatorFactory(INTERPOLATION_METHOD, xyz_rest, coeff_dict).get_interpolator()

If not nearest neighbor interpolation, visualize the triangulation scheme

In [20]:
if INTERPOLATION_METHOD in ["linear", "ct"]:
    fig, ax = plt.subplots(1, 1, dpi=150)
    ax.triplot(xyz_rest[:, 0], xyz_rest[:, 1], interpolator.tri.simplices.copy(), color="black", ls="-", lw=0.8)
    ax.set_aspect("equal")
    ax.set_xlabel(r"$x$ [mm]")
    ax.set_ylabel(r"$y$ [mm]")
    fig.suptitle("Delaunay triangulation")
    fig.show()

For linear interpolation, visualize weights

In [ ]:
if INTERPOLATION_METHOD == "linear" and N_samples == 9:
    nx, ny = (100, 100)
    grid_lims = (-40, 40)
    x = np.linspace(grid_lims[0], grid_lims[1], nx)
    y = np.linspace(grid_lims[0], grid_lims[1], ny)
    # xv, yv = np.meshgrid(x, y, sparse=False)

    model_weights = np.zeros((N_models, nx, ny))
    for i in range(nx):
        for j in range(ny):
            p = [x[i], y[j]]
            simplex = interpolator.tri.find_simplex(p)
            if simplex == -1:
                # extrapolation would be necessary
                model_weights[:, j, i] = np.nan
            else:
                b = interpolator.tri.transform[simplex, :2] @ (p - interpolator.tri.transform[simplex, 2])
                c = np.r_[b, 1 - b.sum()]
                point_idx = interpolator.tri.simplices[simplex]
                model_weights[point_idx, j, i] = c
    fig, axs = plt.subplots(3, 3, figsize=(10, 10), dpi=100, sharey=True, sharex=True, constrained_layout=True)
    im = None
    axs = axs.ravel()
    for i, model in enumerate(model_names):
        ax = axs[i]
        im = ax.contourf(x, y, model_weights[i, :, :],
                        vmin=0, vmax=1, levels=100) # cmap=mpl.colormaps[colormaps[i]])
        # ax.set_title(model)
        ax.set_xlabel(r"$x$ [mm]")
        ax.set_aspect("equal")
    for ax in axs[[0, 3, 6]]:
        ax.set_ylabel(r"$y$ [mm]")
    fig.colorbar(im, ax=axs.ravel().tolist(), fraction=0.03, aspect=35)
    plt.show()

ValueError: wrong dimensionality in xi

Compute observables (delay embedding)

In [ ]:
assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)

Predict open-loop trajectories using the interpolated adiabatic SSM model

In [ ]:
# test_results = {}
# test_trajectory_folders = ['open-loop_circle']
# test_trajectories = []
# for test_traj in test_trajectory_folders:
#     traj_dir = join(robot_dir, "dataCollection", test_traj)
#     (t, z), u = utils.import_pos_data(data_dir=traj_dir,
#                                       rest_file=rest_file,
#                                       output_node=TIP_NODE, return_inputs=True, traj_index=0)
#     # z = (z.T - q_eq).T
#     # u = (u.T - u_eq).T
#     y = assemble_observables(z)
#     print("y.shape:", y.shape)
#     print("u.shape:", u.shape)

#     test_trajectories.append({
#             'name': test_traj,
#             't': t,
#             'z': z,
#             'u': u,
#             'y': y
#         })

In [ ]:
# from scipy.interpolate import interp1d
# from scipy.integrate import solve_ivp

# # global t_prev
# # t_prev = 0

# def advect_adiabaticRD_with_inputs_RK45(t, y0, u, y_target):
#     uInterpFun = interp1d(t, u, axis=1, fill_value="extrapolate")
#     yTargetInterpFun = interp1d(t, y_target, axis=1, fill_value="extrapolate")
#     def R_t(t, x):
#         # global t_prev
#         # print("t:", t, ", t_prev:", t_prev)
#         x = np.atleast_2d(x)
#         # y_target_prev = yTargetInterpFun(t_prev)
#         y_target = yTargetInterpFun(t)
#         # y_bar_prev = np.tile(interpolate_coeffs(y_target_prev[-3:-1], 'q_bar'), 1 + N_DELAY)
#         y_bar = np.tile(interpolate_coeffs(y_target[-3:-1], 'q_bar'), 1 + N_DELAY)
#         # y = ((interpolate_coeffs(y_target_prev[-3:-1], 'W') @ utils.phi(x, SSMOrder)).T + y_bar_prev).T
#         # x[i] = V[i]^T @ (y[i] - y_bar[i])
#         # x = interpolate_coeffs(y_target[-3:-1], 'V').T @ (y.T - y_bar).T
#         u_bar = interpolate_coeffs(y_target[-3:-1], 'u_bar')
#         xdot = (interpolate_coeffs(y_target[-3:-1], 'R') @ utils.phi(x, ROMOrder))# .T + # interpolate_coeffs(y_target[-3:-1], 'B') @ (uInterpFun(t) - u_bar)).T
#         # print(x)
#         # t_prev = t
#         return xdot
#     # solve IVP of reduced dynamics using open-loop inputs
#     x0 = interpolate_coeffs(y0[-3:-1], 'V').T @ (y0.T - np.tile(interpolate_coeffs(y0[-3:-1], 'q_bar'), 1 + N_DELAY))
#     print("x0:", x0)
#     sol = solve_ivp(R_t,
#                     t_span=[t[0], t[-1]],
#                     t_eval=t,
#                     y0=x0,
#                     method="RK45",
#                     vectorized=True,
#                     rtol=1e-3,
#                     atol=1e-3)
#     # resulting (predicted) open-loop trajectory in reduced coordinates
#     xTraj = sol.y
#     yTraj = np.zeros_like(y_target)
#     for i in range(xTraj.shape[1]):
#         yTraj[:, i] = (interpolate_coeffs(y_target[-3:-1, i], 'W') @ utils.phi(xTraj[:, i].reshape(-1, 1), SSMOrder)).T + np.tile(interpolate_coeffs(y_target[-3:-1, i], 'q_bar'), 1 + N_DELAY)
#     return t, yTraj

In [ ]:
# traj = test_trajectories[0]
# print(traj['name'])
# t, x, y_pred, xdot, y_bar, u_bar, weights = advect_adiabaticRD_with_inputs(traj['t'], traj['y'][:, 0], traj['u'], traj['y'], know_target=True)
# # slice_traj = np.s_[:]
# # t, y_pred = advect_adiabaticRD_with_inputs_RK45(traj['t'][slice_traj], traj['y'][:, 0], traj['u'][:, slice_traj], y_target=traj['y'][:, slice_traj])

In [ ]:
# plt.close('all')
# # Trajectory xyz vs. t (predicted vs. actual)
# axs = plot.traj_xyz_txyz(t,
#                             y_pred[0, :], y_pred[1, :], y_pred[2, :],
#                             show=False)
# axs = plot.traj_xyz_txyz(traj['t'],
#                         traj['z'][0, :], traj['z'][1, :], traj['z'][2, :],
#                         color="tab:orange", axs=axs, show=False)
# axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
# for ax in axs:
#     ax.set_ylim(-35, 35)
# plt.suptitle(r"Trajectory (predicted vs. actual)")
# plt.tight_layout()
# plt.show()

# # Trajectory 2D (predicted vs. actual)
# ax = plot.traj_2D_xy(y_pred[0, :], y_pred[1, :],
#                       show=False)
# ax = plot.traj_2D_xy(traj['z'][0, :], traj['z'][1, :],
#                         color="tab:orange", ax=ax, show=False)
# ax.set_xlim(-35, 35)
# ax.set_ylim(-35, 35)
# ax.legend(["Predicted trajectory", "Actual trajectory"])
# plt.suptitle(r"Trajectory (predicted vs. actual)")
# plt.show()

# # weights
# axs = plot.adiabatic_model_weights(t, weights, model_names)

# # psi_bar
# axs = plot.traj_xyz_txyz(t,
#                         y_bar[0, :], y_bar[1, :], y_bar[2, :],
#                         xyz_names=[r"$\bar{\psi}_1(t)$", r"$\bar{\psi}_2(t)$", r"$\bar{\psi}_3(t)$"],
#                         show=False, rotate_yticks=True)
# plt.suptitle(r"$\bar{\psi}(t)$")
# plt.show()

# # xi_dot
# axs = plot.traj_xyz_txyz(t,
#                         xdot[0, :], xdot[1, :], xdot[2, :],
#                         xyz_names=[r"$\dot{\xi}_1(t)$", r"$\dot{\xi}_2(t)$", r"$\dot{\xi}_3(t)$"],
#                         show=False, rotate_yticks=True)
# # for ax in axs:
# #     ax.set_ylim(-100, 100)
# plt.suptitle(r"$\dot{\xi}(t)$")
# plt.show()

# # xi
# axs = plot.traj_xyz_txyz(t,
#                         x[0, :], x[1, :], x[2, :],
#                         xyz_names=[r"$\xi_1(t)$", r"$\xi_2(t)$", r"$\xi_3(t)$"],
#                         show=False, rotate_yticks=True)
# # for ax in axs:
# #     ax.set_ylim(-100, 100)
# plt.suptitle(r"$\xi(t)$")
# plt.show()


In [ ]:
# plt.close('all')

## Test "autonomous" Adiabatic SSM

Ideal target trajectory: circle starting at $\varphi=\frac{\pi}{2}$

In [ ]:
z_eq = np.array([3.54595786, 0.30603326, -195.61577293])

M = 1
T = 9 # 10
N = 900 # 1000
radius = 30.
t_target = np.linspace(0, M * T, M * N + 1)
th = np.linspace(0, M * 2 * np.pi, M * N + 1) + np.pi/2
zf_target = np.tile(z_eq, (M * N + 1, 1))
# zf_target = np.zeros((M*N+1, 6))
zf_target[:, 0] += radius * np.cos(th)
zf_target[:, 1] += radius * np.sin(th)
# zf_target[:, 2] += -np.ones(len(t)) * 20
# print(zf_target[0, :].shape)
# idle = np.repeat(np.atleast_2d(zf_target[0, :]), int(1/0.01), axis=0)
# print(idle.shape)
# zf_target = np.vstack([idle, zf_target])
# print(zf_target.shape)
# t = np.linspace(0, M * 10, M * 1000 + 1)

z_target = zf_target - z_eq
y_target = assemble_observables(z_target.T)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)
ax.plot(z_target[:, 0], z_target[:, 1], color="k", ls="--", lw=2)
ax.plot(z_target[0, 0], z_target[0, 1], marker="D", markersize=10, ls="", color="k")
ax.grid()
fig.show()

Track an ideal circle using a previous controller (SSMR).

In [ ]:
traj_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/autonomous_ASSM_tests"
(t, z), u = utils.import_pos_data(data_dir=join(traj_dir, "circle_ideal_using-ssmr-controller"),
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True)
y = assemble_observables(z)
print("y.shape:", y.shape)
print("z.shape:", z.shape)
print("u.shape:", u.shape)

y.shape: (15, 1102)
z.shape: (3, 1102)
u.shape: (8, 1102)


Smoothen inputs

In [ ]:
from copy import deepcopy

In [ ]:
def smooth(weights, arr):
   return np.convolve(weights/weights.sum(), arr)

In [ ]:
u_smooth = deepcopy(u)
smooth_window_size = 20
for i in range(u.shape[0]):
    u_smooth[i, :] = smooth(np.hanning(smooth_window_size), u[i, :])[:-smooth_window_size+1]

Plot the ideal trajectory

In [ ]:
plot.inputs(t, u)
plot.inputs(t, u_smooth)

In [ ]:
plt.close('all')

Simulate two trajectories from this: one using the smoothened inputs ($z_0$); one using the smoothened inputs but with a perturbation before starting the trajectory ($z_0+\varepsilon$)

In [ ]:
with open(join(traj_dir, "u_ideal.pkl"), "wb") as f:
    model = pickle.dump(u_smooth, f)

In [ ]:
# add perturbation to idle inputs
perturbation = np.array([50, 0, 0, 0, 50, 0, 0, 0])
u_perturbed = deepcopy(u_smooth)
u_perturbed[:, 100:200] += perturbation.reshape(-1, 1)
with open(join(traj_dir, "u_perturbed.pkl"), "wb") as f:
    model = pickle.dump(u_perturbed, f)

In [ ]:
plot.inputs(t, u_perturbed)

Do open-loop simulations using these two input sequences in SOFA

...

Import the resulting trajectories:
- unperturbed: should be close to ideal
- perturbed: ground truth; compare against advection

In [ ]:
# unperturbed
(t_unperturbed, z_unperturbed), u_unperturbed = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_ideal"),
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True, t_in=2)
y_unperturbed = assemble_observables(z_unperturbed)
print("y.shape:", y_unperturbed.shape)
print("z.shape:", z_unperturbed.shape)
print("u.shape:", u_unperturbed.shape)
# perturbed
(t_perturbed, z_perturbed), u_perturbed = utils.import_pos_data(data_dir=join(traj_dir, "OL_sim_perturbed"),
                                  rest_file=rest_file,
                                  output_node=TIP_NODE, return_inputs=True, t_in=2)
y_perturbed = assemble_observables(z_perturbed)
print("y.shape:", y_perturbed.shape)
print("z.shape:", z_perturbed.shape)
print("u.shape:", u_perturbed.shape)

y.shape: (15, 902)
z.shape: (3, 902)
u.shape: (8, 902)
y.shape: (15, 902)
z.shape: (3, 902)
u.shape: (8, 902)


Plot: (a) ideal target, (b) unperturbed open-loop sim, (c) perturbed open-loop sim

In [ ]:
plt.close("all")
# Trajectories xyz vs. t
axs = plot.traj_xyz_txyz(t_unperturbed,
                         y_unperturbed[0, :], y_unperturbed[1, :], y_unperturbed[2, :],
                         show=False)
axs = plot.traj_xyz_txyz(t_perturbed,
                         y_perturbed[0, :], y_perturbed[1, :], y_perturbed[2, :],
                         color="tab:orange", axs=axs, show=False)
axs = plot.traj_xyz_txyz(t_target,
                         z_target[:, 0], z_target[:, 1], z_target[:, 2],
                         color="k", ls="--", axs=axs, show=False)
axs[-1].legend(["Unperturbed", "Perturbed", "Target"])
plt.suptitle(r"Trajectories")
plt.tight_layout()
plt.show()

# Trajectories 2D (predicted vs. actual)
ax = plot.traj_2D_xy(y_unperturbed[0, :], y_unperturbed[1, :],
                     show=False)
ax = plot.traj_2D_xy(y_perturbed[0, :], y_perturbed[1, :],
                     color="tab:orange", ax=ax, show=False)
ax = plot.traj_2D_xy(z_target[:, 0], z_target[:, 1],
                     color="k", ls="--", ax=ax, show=False)
ax.legend(["Unperturbed", "Perturbed", "Target"])
plt.suptitle(r"Trajectories")
plt.show()

Then advect the perturbed case using ROM (SSMR, ASSMR)

In [ ]:
t, x, y_pred, xdot, y_bar, u_bar, weights = utils.advect_adiabaticRD_with_inputs(t_unperturbed, y_perturbed[:, 0], u_unperturbed, y_unperturbed, interpolator=interpolator, know_target=True, ROMOrder=ROMOrder)

Compute RSME

In [ ]:
assert len(t) == len(t_perturbed)
print(y_pred.shape)
print(y_perturbed.shape)
rmse = np.sum(np.sqrt(np.mean((y_pred[:3, :-1] - y_perturbed[:3, :-1])**2, axis=0))) / len(t)
print(f"RMSE = {rmse:.4f}")

(15, 902)
(15, 902)
RMSE = nan


Save the predicted trajectory for these model settings. Use later to plot comparison.

In [ ]:
with open(join(traj_dir, "predicted_trajectories", f"y_pred_{INTERPOLATION_METHOD}_ROMOrder={ROMOrder}.pkl"), "wb") as f:
    model = pickle.dump(y_pred, f)

In [ ]:
plt.close('all')
# Trajectory xyz vs. t (predicted vs. actual)
axs = plot.traj_xyz_txyz(t,
                         y_pred[0, :], y_pred[1, :], y_pred[2, :],
                         show=False)
axs = plot.traj_xyz_txyz(t_perturbed,
                        y_perturbed[0, :], y_perturbed[1, :], y_perturbed[2, :],
                        color="tab:orange", axs=axs, show=False)
axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.tight_layout()
plt.show()

# Trajectory 2D (predicted vs. actual)
ax = plot.traj_2D_xy(y_pred[0, :], y_pred[1, :],
                     show=False)
ax = plot.traj_2D_xy(y_perturbed[0, :], y_perturbed[1, :],
                     color="tab:orange", ax=ax, show=False)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.show()

Plot comparison of different model settings

In [ ]:
# plt.close('all')
# display_names = {
#     "origin_only": "SSMR (origin only)",
#     "linear": "ASSMR (linear)",
#     "ct": "ASSMR (Clough-Tocher)",
#     "nn": "ASSMR (nearest neighbor)"
# }
# colors = {
#     "origin_only": "tab:blue",
#     "linear": "tab:orange",
#     "ct": "tab:green",
#     "nn": "tab:purple"
# }

# for rom_order in [1, 3]:
#     axs = plot.traj_xyz_txyz(t_perturbed,
#                              y_perturbed[0, :], y_perturbed[1, :], y_perturbed[2, :],
#                             show=False, color="k", ls="--", label="True trajectory")
#     ax = plot.traj_2D_xy(y_perturbed[0, :], y_perturbed[1, :],
#                          show=False, color="k", ls="--", label="True trajectory")
#     for interpolation_method in ["origin_only", "linear", "ct", "nn"]:
#         # Trajectory xyz vs. t (predicted vs. actual)
#         with open(join(traj_dir, "predicted_trajectories", f"y_pred_{interpolation_method}_ROMOrder={rom_order}.pkl"), "rb") as f:
#             y_pred = pickle.load(f)
#         axs = plot.traj_xyz_txyz(t,
#                                 y_pred[0, :], y_pred[1, :], y_pred[2, :],
#                                 color=colors[interpolation_method], axs=axs, show=False, label=display_names[interpolation_method])
#         # Trajectory 2D (predicted vs. actual)
#         ax = plot.traj_2D_xy(y_pred[0, :], y_pred[1, :],
#                              color=colors[interpolation_method], ax=ax, show=False, label=display_names[interpolation_method])
#     # axs[-1].legend()
#     # axs[-1].get_figure().suptitle(rf"Trajectories (true vs. predicted) -- ROMOrder={rom_order}")
#     axs[-1].get_figure().tight_layout()
#     axs[-1].get_figure().savefig(join(traj_dir, f"xyz_vs_t_ROMOrder={rom_order}.png"), bbox_inches='tight')
#     axs[-1].get_figure().show()
#     ax.legend()
#     # ax.get_figure().suptitle(rf"Trajectories (true vs. predicted) -- ROMOrder={rom_order}")
#     ax.get_figure().savefig(join(traj_dir, f"x_vs_y_ROMOrder={rom_order}.png"), bbox_inches='tight')
#     ax.get_figure().show()